# Analysis on the students attenton


First of all, the necessary imports


In [ ]:
import pandas as pd
import glob
import random

Then reading the 80 csv files, getting the time etiquete, which is not set, join all the csv files and sort them by time. Important to mention that this is NOT part of the model. Only for data analysis and estimations before the model

In [ ]:
# Read the debugging-performance file apart so that we can add a column to the actual data with the score 
debugging = pd.read_csv("applied-data-science-dataset/debugging-performance.csv")
# Make our way easier to handle which participant have which performance
performance_dict = dict(zip(debugging['participant'], debugging['performance']))

# List to store the individual dataframes
dataframes = []

# Keep in mind that the debugging performance file is not taken into account 
for i in range(1, 81):
    if i < 10:
        file = f"applied-data-science-dataset/P0{i}.csv"
    else:
        file = f"applied-data-science-dataset/P{i}.csv"
    
    df = pd.read_csv(file)
    
    df.rename(columns={'Unnamed: 0': 'Time'}, inplace=True)
    
    name = f'P{i:02}'
    df['Source'] = name
    df['Performance'] = performance_dict[name]
    
    # Append the dataframe to the list
    dataframes.append(df)

# Now that all dataframes are loaded and sorted by 'Time', concatenate them
# This time, concatenate across columns for each 'Time' value
combined_df = pd.concat(dataframes, ignore_index=True)

combined_df.sort_values(by=['Time', 'Source'], inplace=True)

combined_df.drop(columns=['Source'], inplace=True)

# Reset index after sorting to maintain continuous indexing
combined_df.reset_index(drop=True, inplace=True)

# Save the final combined dataframe to a CSV file
combined_df.to_csv("combined_dataset.csv", index=False)


Now we can focus on preprocessing data: Since the column of "Time" isn't set propertly, we can fix that manually, then, pick our training and testing data (only once) by random, the distribution would be 60-20 since is the closest to 2/3 - 1/3 that could work for our model

## DO NOT EXECUTE THIS CODE, THIS ALREADY RANDOMLY GENERATED THE TRAIN AND TEST DATA
(In fact I'm going to comment it so that all the cells can be executed) Remove the comment if needed other training data

In [ ]:
"""
# This line could be not necessary, but this way, cells can be executed independently
file_paths = glob.glob("applied-data-science-dataset/P*.csv")
# Read the debugging-performance file apart so that we can add a column to the actual data with the score 
debugging = pd.read_csv("applied-data-science-dataset/debugging-performance.csv")
# Make our way easier to handle which participant have which performance
performance_dict = dict(zip(debugging['participant'], debugging['performance']))

# Pick randomly 60 students
training_files = random.sample(file_paths, 60)

testing_files = list(set(file_paths) - set(training_files))

training_files.sort()
testing_files.sort()


# Function to load, rename the time column, and return the DataFrame
def load_and_prepare_csv(file):
    df = pd.read_csv(file)
    df.rename(columns={'Unnamed: 0': 'Time'}, inplace=True) 
    name = f'P{int(file[30:32]):02}'
    df['Source'] = name
    df['debuggingPerformance'] = performance_dict[name]
    return df

# Concatenate the training CSVs
training_dfs = [load_and_prepare_csv(file) for file in training_files]
training_df = pd.concat(training_dfs, ignore_index=True)

# Concatenate the testing CSVs
testing_dfs = [load_and_prepare_csv(file) for file in testing_files]
testing_df = pd.concat(testing_dfs, ignore_index=True)

# Sort both datasets by 'Time' and by the students name (added that column so it's easier to identify them)
training_df.sort_values(by=['Time', 'Source'], inplace=True)
testing_df.sort_values(by=['Time', 'Source'], inplace=True)

training_df.to_csv("model_data/training_dataset.csv", index=False)
testing_df.to_csv("model_data/testing_dataset.csv", index=False)
"""

Once the training data and testing data are separated, we can make the rest of the pre-processing

In [ ]:
training_data = pd.read_csv("model_data/training_dataset.csv")
test_data = pd.read_csv("model_data/testing_dataset.csv")

# Now, the point of preprocessing, would be to fill the missing values, in our case with the mean
def fill_missing_values(data):
    return data.fillna(data.mean())
# But since none of the datasets have missing values, theres hardly a point in doing this:
training_data = fill_missing_values(training_data)
test_data = fill_missing_values(test_data)

# The same could be said about scalating the values, but other than the actual time
# (which we dont want to scalate) the values are already scalated.

def scalate_data(data, columns_to_scale, scaler):
    df[columns_to_scale] = scaler.fit_transform(df[columns_to_scale])
    return df

# And finally, depending on which metric we are going to choose for training 
# (which for example, can be the debugging performance) and split the data accordingly

X_train, y_train = training_data.drop(column=['debuggingPerformance']), training_data['debuggingPerformance']
 
X_test, y_test  = test_data.drop(column=['debuggingPerformance']), test_data['debuggingPerformance']




